# Prophet - Predecir el valor de Bitcoin


Prophet es una de las librerías más avanzadas para predecir series temporales desarrollada por Facebook. Te enseñaremos a como entrenar un modelo con Prophet, a optimizarlo y a utilizarlo para realizar predicciones futuras. En este ejercicio vamos a practicar a predecir el valor de Bitcoin, una criptomoneda. Es la criptomoneda que le ha marcado el camino a todas las demás que llegaron después utilizando su tecnología.  

<hr/>
<div class="alert alert-success alertsuccess" style="margin-top: 20px">
[Tip]: Para ejecutar el código de Python en la celda de código a continuación, haz clic en la celda para seleccionarla y presiona <kbd>Shift</kbd> + <kbd>Enter</kbd>.
</div>
<hr/>


In [4]:
#Dependencias de Prophet
!pip install pystan==2.19.1.1 --quiet
!pip install cmdstanpy --quiet
# Instalamos el modelo predictivo Prophet
!pip install fbprophet --quiet
# De aquí vamos a descargar los datos
!pip install yfinance --quiet

In [2]:
#Importamos las dependencias
import pandas as pd
import yfinance as yf
from datetime import datetime
from datetime import timedelta
import plotly.graph_objects as go
from fbprophet import Prophet
from fbprophet.plot import plot_plotly, plot_components_plotly
import warnings

warnings.filterwarnings('ignore')

pd.options.display.float_format = '${:,.2f}'.format

<h3 id="version">Cargar histórico de datos</h3>


<p>
    Para descargarnos la serie temporal de BTC lo vamos a hacer desde Yahoo Finance. En concreto vamos a descargar el histórico desde 2016 hasta la actualidad.
</p>



In [7]:
# Fecha de inicio es un string 2016-01-01 y fecha de fin (hoy) en formato texto '%Y-%m-%d'
today = '2023-03-23'
start_date = '2016-01-01'
# Descargar el dataframe
rub_df = yf.download('RUBGBP=X',start_date, today).reset_index()

rub_df.tail()

[*********************100%***********************]  1 of 1 completed


,Date,Open,High,Low,Close,Adj Close,Volume
1876,2023-03-16,$0.01,$0.01,$0.01,$0.01,$0.01,0
1877,2023-03-17,$0.01,$0.01,$0.01,$0.01,$0.01,0
1878,2023-03-20,$0.01,$0.01,$0.01,$0.01,$0.01,0
1879,2023-03-21,$0.01,$0.01,$0.01,$0.01,$0.01,0
1880,2023-03-22,$0.01,$0.01,$0.01,$0.01,$0.01,0


In [8]:
# Validamos que no hay datos vacíos
rub_df.isnull().sum()

Date         0
Open         0
High         0
Low          0
Close        0
Adj Close    0
Volume       0
dtype: int64

In [22]:
# TAREA: Filtramos el dataframe para quedarnos solo las columnas "Date" y "Open"
rub_df = rub_df[['Open','Date']]

# Date será la fecha del valor y Open el valor del BTC en ese momento

# Valores que espera el prophet
# Eje X: Indíce llamado ds
# Eje Y: Serie a predecir llamada y
# TAREA HECHA: Crear un diccionario con clave "Date" y valor "ds"
# Y otra clave "Open" y valor "y"
new_names = {
    "Date": "ds", 
    "Open": "y",
}

# TAREAUtiliza el diccionario para renombrar las columnas de df
df = rub_df.rename(columns=new_names)
df

,y,ds
0,$0.01,2016-01-01
1,$0.01,2016-01-04
2,$0.01,2016-01-05
3,$0.01,2016-01-06
4,$0.01,2016-01-07
...,...,...
1876,$0.01,2023-03-16
1877,$0.01,2023-03-17
1878,$0.01,2023-03-20
1879,$0.01,2023-03-21


In [23]:
#Ejecuta la siguiente línea, si lo has hecho bien hasta aquí no dará error
df['ds'] = df['ds'].dt.tz_localize(None)

In [24]:
df.tail()

,y,ds
1876,$0.01,2023-03-16
1877,$0.01,2023-03-17
1878,$0.01,2023-03-20
1879,$0.01,2023-03-21
1880,$0.01,2023-03-22


In [25]:
# Crear la gráfica del precio de apertura
#Asigna a la variable x la columna ds
x = df["ds"]
#Asigna a la variable y la columna y
y = df["y"]

fig = go.Figure()

fig.add_trace(go.Scatter(x=x, y=y))

# Le ponemos el título a la gráfica
titulo = "Tabla rublo"
fig.update_layout(
    title_text=titulo,
)

fig.update_layout(
    xaxis=dict(
        rangeselector=dict(
            buttons=list(
                [
                    dict(count=1, label="1m", step="month", stepmode="backward"),
                    dict(count=6, label="6m", step="month", stepmode="backward"),
                    dict(count=1, label="YTD", step="year", stepmode="todate"),
                    dict(count=1, label="1a", step="year", stepmode="backward"),
                    dict(step="all"),
                ]
            )
        ),
        rangeslider=dict(visible=True),
        type="date",
    )
)

<h3 id="version">Entrenar y predecir el modelo</h3>


<p>
    Ahora que has visto en la gráfica de que datos partimos. Inicializamos el Prophet y lo entramos pasandole el dataframe
</p>



In [27]:
#TAREA: Inicializa Prophet en la variable m con seasonality_mode en modo "multiplicative"
M=Prophet(seasonality_mode ='multiplicative')

In [30]:
#TAREA: Entrena el modelo m con el dataframe de datos df
M.fit(df)

INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.


<p>
    Le decimos al modelo m los días en futuro que queremos predecir, en este caso el próximo año.
</p>



In [34]:
# TAREA: generar un dataframe con los días que tenemos y los 365 días siguientes a hoy para predecirlo
# PISTA: puedes usar la función del prophet make_future_dataframe()
future = M.make_future_dataframe(periods=365)
future.tail()

,ds
2241,2024-03-17
2242,2024-03-18
2243,2024-03-19
2244,2024-03-20
2245,2024-03-21


In [35]:
# En el caso que hayas elegido un valor o fondo tradicional, es decir, que no opere los fines
# de semana, descomenta la siguiente línea:

future = future[ future['ds'].dt.dayofweek < 5 ] # Nos elimina los fines de semana de 'ds' para no predecirlos
future

,ds
0,2016-01-01
1,2016-01-04
2,2016-01-05
3,2016-01-06
4,2016-01-07
...,...
2239,2024-03-15
2242,2024-03-18
2243,2024-03-19
2244,2024-03-20


<p>
    Al modelo entrenado le pasamos el dataframe a predecir.
</p>



In [36]:
#Utiliza el modelo m para predecir el dataframe future
forecast = M.predict(future)

In [37]:
forecast[['ds', 'yhat', 'yhat_lower', 'yhat_upper']].tail()

,ds,yhat,yhat_lower,yhat_upper
2137,2024-03-15,$0.01,$-0.00,$0.03
2138,2024-03-18,$0.01,$-0.00,$0.03
2139,2024-03-19,$0.01,$-0.00,$0.03
2140,2024-03-20,$0.01,$-0.00,$0.03
2141,2024-03-21,$0.01,$-0.00,$0.03


<p>
    Gráfica con el modelo entrenado y los valores de un año en futuro predecidos.
</p>



In [39]:
next_day = (datetime.today() + timedelta(days=1)).strftime('%Y-%m-%d')
forecast[forecast['ds'] == next_day]['yhat'].item()
plot_plotly(M, forecast)

In [40]:
plot_components_plotly(M, forecast)